In [12]:
import librosa
import soundfile
import os, glob, subprocess
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from pandas import DataFrame
import warnings
warnings.filterwarnings('ignore')

In [5]:
# import sys
# !{sys.executable} -m pip install librosa

In [13]:
input_file_path = 'c:/fomc/fVUGwgLTzbw_000.mp4'
output_file_path = os.path.join(os.path.splitext(input_file_path)[0] + ".wav")

os.path.join(os.path.splitext(input_file_path)[0] + ".wav")

'c:/fomc/fVUGwgLTzbw_000.wav'

In [5]:
# Use FFmpeg to extract audio from the MP4 file and save as WAV
subprocess.run(["ffmpeg", "-i", input_file_path, "-vn", "-acodec", "pcm_s16le", "-ar", "44100",
                "-ac", "1", output_file_path], check=True)

CompletedProcess(args=['ffmpeg', '-i', 'c:/fomc/fVUGwgLTzbw_000.mp4', '-vn', '-acodec', 'pcm_s16le', '-ar', '44100', '-ac', '1', 'c:/fomc/fVUGwgLTzbw_000.wav'], returncode=0)

In [14]:
def extract_feature(file_name):
    '''
        Extract the following features
            - MFCC (mfcc)
            - Chroma (chroma)
            - MEL Spectrogram Frequency (mel)
            - Contrast (contrast)
            - Tonnetz (tonnetz)
        Not sure if others are useful: spectral_centroid, flatness, rolloff, etc.
    '''
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        #Short-time Fourier transform
        stft = np.abs(librosa.stft(X))
        result = np.array([])
        #mfcc
        mfccs = np.mean(librosa.feature.mfcc(S=stft, sr=sample_rate, n_mfcc=40).T, axis=0)
        mfccs_df = DataFrame(mfccs.reshape(-1, len(mfccs)))
        mfccs_df.columns=['mfccs'+str(i) for i in range(0,len(mfccs))]
        #chroma
        chroma = np.mean(librosa.feature.chroma_stft(y=X, sr=sample_rate).T,axis=0)
        chroma_df = DataFrame(chroma.reshape(-1, len(chroma)))
        chroma_df.columns=['chroma'+str(i) for i in range(0,len(chroma))]
        #mel
        mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T,axis=0)
        mel_df = DataFrame(mel.reshape(-1, len(mel)))
        mel_df.columns=['mel'+str(i) for i in range(0,len(mel))]
#         #contrast
#         contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=sample_rate).T,axis=0)
#         contrast_df = DataFrame(contrast.reshape(-1, len(contrast)))
#         contrast_df.columns=['contrast'+str(i) for i in range(0,len(contrast))]
#         #tonnetz
#         tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(X), sr=sample_rate).T,axis=0)
#         tonnetz_df = DataFrame(tonnetz.reshape(-1, len(tonnetz)))
#         tonnetz_df.columns=['tonnetz'+str(i) for i in range(0,len(tonnetz))]
    return mfccs_df, chroma_df, mel_df #, contrast_df, tonnetz_df

In [15]:
mfccs, chroma, mel=extract_feature(output_file_path) #, contrast, tonnetz
rows = pd.concat([mfccs, chroma, mel], axis=1)
print(rows)

     mfccs0    mfccs1    mfccs2    mfccs3    mfccs4    mfccs5    mfccs6  \
0  6.792516  8.580993  6.653436  5.246355  4.367994  3.696401  3.208529   

     mfccs7    mfccs8    mfccs9  ...        mel118        mel119  \
0  2.880311  2.729712  2.688181  ...  8.201210e-07  1.015757e-08   

         mel120        mel121        mel122        mel123        mel124  \
0  2.843604e-09  2.815626e-09  2.800723e-09  2.791057e-09  2.773310e-09   

         mel125        mel126        mel127  
0  2.765620e-09  2.761521e-09  2.772458e-09  

[1 rows x 180 columns]
